## Selector Group Chat  

AutoGen offers several predefined teams, in addition to the `RoundRobinGroupChat`. 
One example is the `Selector Group Chat`, where all agents share the same context, and an LLM is responsible for selecting the next "speaker" in the group. 

In [1]:
import sys
sys.path.append("..")

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from model_clients.azure import get_model
from rich import print
import random

model_client = get_model()


Let's start by creating the agent in charge of deciding which agent is in charge to execute (a.k.a the group chat moderator)

In [2]:
customer_agent = AssistantAgent(
    "customer_agent",
    model_client=model_client,
    description="A bank assistant.",
    system_message="""You are a bank assistant.
    Your job is to break down complex tasks into smaller, manageable subtasks.

    Your team members are:
        account_agent: provides account ID
        saving_account_agent: provides saving account balance
        investment_agent: provides investment account balance

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    <agent> : <task>

    After all tasks are complete,  Provide your response in a JSON format."

    ```json{
        "account_id": "<account id>",
        "saving_balance": <saving balance>,
        "investment_balance": <investment balance>,
        "total_balance": <total balance>
    }```

    And, end with "TERMINATE".
    """,
)

Now let's define the tools (aka the functions) that will be used by the agents

In [3]:
# Define the tools used by the agents

balances = {
        "123": 100000.10,
        "456": 200000.20,
        "789": 300000.30,
    }

def get_bank_account_id() -> str:
    """Get the bank account ID."""
    id = random.choice(["123", "456", "789"])
    print(f"[red][TOOL] Bank account ID: {id}[/red]")
    return id

def get_investment_account_balance(bank_account_id: str) -> float:
    """Get the investment balance of an account.

    :param bank_account_id: The account ID.
    :return: The investment balance.
    """    

    balance = balances[bank_account_id]
    print(f"[green][TOOL] Investment balance: {balance}[/green]")
    return balance

def get_saving_account_balance(bank_account_id: str) -> float:
    """Get the saving balance of an account.

    :param bank_account_id: The account ID.
    :return: The saving balance.
    """
    balance = balances[bank_account_id]
    print(f"[cyan][TOOL] Saving balance: {balance}[/cyan]")
    return balance

And finally the agents...

In [4]:
account_agent = AssistantAgent(
    "account_agent",
    model_client=model_client,
    description="An account agent.",
    tools=[get_bank_account_id],
    system_message="""You are an account agent who can provide account ID.
You should always use the tool provided to generate the account balance.
    """,  
)

investment_agent = AssistantAgent(
    "investment_agent",
    model_client=model_client,
    description="An investment account agent.",
    tools=[get_investment_account_balance],
    system_message="""You are an investment account agent who can provide information about the investment account balance.
Look at the chat history to understand the context of the conversation and the account id is in the it. Look for investment account ID.
You should always use the tool provided to generate the account balance.
    """, 
)

saving_account_agent = AssistantAgent(
    "saving_account_agent",
    model_client=get_model(),
    description="A saving account agent.",
    tools=[get_saving_account_balance],
    system_message="""You are a saving account agent who can provide information about the saving account balance.
Look at the chat history to understand the context of the conversation and the account id is in the it. Look for Saving account ID.
You should always use the tool provided to generate the account balance.
    """,  
)

Now, we create the team, this time using a `SelectorGroupChat` consisting of the four agents we just created.  
Please note the following:  

1. We create a special prompt that allows the `customer_agent` (the chat moderator) to decide which agent will be invoked next.  
2. To prevent the conversation from entering an infinite loop, we define two termination conditions:  
   1. When the request is completed, indicated by the presence of **TERMINATE** in the response.  
   2. When the number of exchanged messages exceeds 25.  


In [5]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the
next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

team = SelectorGroupChat(
    [customer_agent, account_agent, saving_account_agent, investment_agent],
    model_client=model_client,
    termination_condition=TextMentionTermination("TERMINATE") | MaxMessageTermination(max_messages=25),
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

It's now time to run the team.

In [ ]:
await team.reset()
result = await Console(
        team.run_stream(
            task="Get the account ID and then get the saving balance "
            "and investment balance. Both saving and investment account have"
            "the same account ID. Sum the balances when they are available."
        )
    )

---------- user ----------
Get the account ID and then get the saving balance and investment balance. Both saving and investment account havethe same account ID. Sum the balances when they are available.
---------- account_agent ----------
[FunctionCall(id='call_cEmxG69CAcqjUeiqo3nftyGZ', arguments='{}', name='get_bank_account_id')]


[TOOL] Bank account ID: 456

---------- account_agent ----------
[FunctionExecutionResult(content='456', name='get_bank_account_id', call_id='call_cEmxG69CAcqjUeiqo3nftyGZ', is_error=False)]
---------- account_agent ----------
456
---------- saving_account_agent ----------


[TOOL] Saving balance: 200000.2

[FunctionCall(id='call_qUZTj9tr4DNOxebTmL2tM6uj', arguments='{"bank_account_id": "456"}', name='get_saving_account_balance')]
---------- saving_account_agent ----------
[FunctionExecutionResult(content='200000.2', name='get_saving_account_balance', call_id='call_qUZTj9tr4DNOxebTmL2tM6uj', is_error=False)]
---------- saving_account_agent ----------
200000.2


[TOOL] Investment balance: 200000.2

---------- investment_agent ----------
[FunctionCall(id='call_KZeqa3UtrdGEctAozZ2WYCMh', arguments='{"bank_account_id":"456"}', name='get_investment_account_balance')]
---------- investment_agent ----------
[FunctionExecutionResult(content='200000.2', name='get_investment_account_balance', call_id='call_KZeqa3UtrdGEctAozZ2WYCMh', is_error=False)]
---------- investment_agent ----------
200000.2
---------- customer_agent ----------
account_agent : Retrieve account ID
saving_account_agent : Retrieve saving account balance using account ID 456
investment_agent : Retrieve investment account balance using account ID 456

Once these tasks are complete, calculate the total balance by summing the saving balance and investment balance.

```json
{
    "account_id": "456",
    "saving_balance": 200000.2,
    "investment_balance": 200000.2,
    "total_balance": 400000.4
}
```

TERMINATE
